In [ ]:
import numpy as np
import pandas as pd
import h5py
from sklearn import svm, model_selection, neighbors, metrics, linear_model, naive_bayes, tree

In [ ]:
"""
Read in the saved train data
"""
g = h5py.File('datasets/train_vectors_tenK.hdf5', 'r')
train_preprocessed_x = np.array(g['train_preprocessed'])
train_preprocessed_y = np.array(pd.read_csv('datasets/train_tenK_vectors_labels.csv', usecols=['Category']))


In [ ]:
"""
Splitting the training data into train, valid and test
"""
train_x, test_x, train_y, test_y = model_selection.train_test_split(train_preprocessed_x, train_preprocessed_y, test_size=0.4, 
                                                                    shuffle=False)
test_x, valid_x, test_y, valid_y = model_selection.train_test_split(test_x, test_y, test_size=0.5, shuffle=False)
train_y = train_y.ravel()
valid_y = valid_y.ravel()
test_y = test_y.ravel()

# SVM

In [ ]:
#Hyper parameter grid to be tested for Linear SVM 
param_grid_svc= {'C': [1*i for i in range(1, 10)], 'tol': [10**(-i*2) for i in range(1, 10)]}

In [ ]:
""""
using grid search to fit the data on best hyper parameters with good f1_score
""""
linSVC = svm.LinearSVC(dual=True)
linSVC = model_selection.GridSearchCV(linSVC, param_grid_svc, scoring='f1_micro', cv=3, verbose=2, n_jobs=2)
linSVC = linSVC.fit(train_x, train_y)

In [ ]:
print(linSVC.best_params_, linSVC.best_score_)

In [ ]:
"""
fitting Linear SVC on the best hyper parameters found from grid search
Using this fit, finding the prediction for the test set and then finding the f1_score of this prediction
""""
linSVC = svm.LinearSVC(tol= linSVC.best_params_['tol'], C=linSVC.best_params_['C'])
linSVC = linSVC.fit(train_x, train_y)

linearsvm_test_pred = linSVC.predict(test_x)

linearsvm_test_f1 = metrics.f1_score(test_y, linearsvm_test_pred, average='micro')

In [ ]:
print(linearsvm_test_f1*100)

# Logistic Regression

In [ ]:
#Hyperparameter grid for the Logistic Regression
param_grid_LR = {'C': [0.000000001, 0.0000001, 0.000001,0.00001, 0.001,0.01,0.1,1,10,100]}

In [ ]:
""""
using grid search to fit the data on best hyper parameters with good f1_score
""""
lr = linear_model.LogisticRegression()
lr = model_selection.GridSearchCV(lr, param_grid_LR, cv=3, scoring='f1_micro', verbose=2, n_jobs=-1)
lr = lr.fit(train_x, train_y)

In [ ]:
print(lr.best_params_, lr.best_score_)

In [ ]:
"""
fitting Logistic Regression on the best hyper parameters found from grid search
Using this fit, finding the prediction for the test set and then finding the f1_score of this prediction
""""
lr = linear_model.LogisticRegression(C=lr.best_params_['C'])
lr =lr.fit(train_x, train_y)

lr_test_pred = lr.predict(test_x)

lr_test_f1 = metrics.f1_score(test_y, lr_test_pred, average='micro')

In [ ]:
print(lr_test_f1*100)

# Multinomial Naive Bayes

In [ ]:
#Hyperparamter for the Multinomial Naive Bayes
param_grid_NB = {'alpha':[0.0001, 0.001, 0.01, 1, 10, 20, 30, 50, 65,100]}

In [ ]:
""""
using grid search to fit the data on best hyper parameters with good f1_score
""""
mNB = naive_bayes.MultinomialNB()
mNB = model_selection.GridSearchCV(mNB, param_grid_NB, cv=2, scoring='f1_micro', verbose=2, n_jobs=-1)
mNB = mNB.fit(train_x, train_y)

In [ ]:
print(mNB.best_params_, mNB.best_score_)

In [ ]:
"""
fitting Multinomial Naive Bayes on the best hyper parameters found from grid search
Using this fit, finding the prediction for the test set and then finding the f1_score of this prediction
""""
mNB = naive_bayes.MultinomialNB(alpha=mNB.best_params_['alpha'])
mNB =mNB.fit(train_x, train_y)

mNB_test_pred = mNB.predict(test_x)

mNB_test_f1 = metrics.f1_score(test_y, mNB_test_pred, average='micro')


In [ ]:
print(mNB_test_f1*100)

# Decision Trees

In [ ]:
#Hyperparamter grid for Decision Trees
param_grid_DT = {'criterion': ['gini', 'entropy'], 'max_depth': [10, 8, 6, 4, 2], 'min_samples_split': [600, 400,200, 100], 
                 'min_samples_leaf':[600, 400, 200, 100], 'max_features': [10, 50, 100, 200, 500, 900]}

In [ ]:
""""
using grid search to fit the data on best hyper parameters with good f1_score
""""
decisionTr = tree.DecisionTreeClassifier()
decisionTr = model_selection.GridSearchCV(decisionTr, param_grid_DT, cv=2, scoring='f1_micro', verbose=2, n_jobs=-1)
decisionTr = decisionTr.fit(train_x, train_y)

In [ ]:
print(decisionTr.best_params_, decisionTr.best_score_)

In [ ]:
"""
fitting Decision Trees on the best hyper parameters found from grid search
Using this fit, finding the prediction for the test set and then finding the f1_score of this prediction
""""
decisionTr = tree.DecisionTreeClassifier(criterion=decisionTr.best_params_['criterion'], 
            max_depth=decisionTr.best_params_['max_depth'], min_samples_leaf=decisionTr.best_params_['min_samples_leaf'], 
            max_features=decisionTr.best_params_['max_features'], min_samples_split=decisionTr.best_params_['min_samples_split'])
mNB =mNB.fit(train_x, train_y)

mNB_test_pred = mNB.predict(test_x)

mNB_test_f1 = metrics.f1_score(test_y, mNB_test_pred, average='micro')


In [ ]:
print(mNB_test_f1*100)

# Kaggle test set

In [ ]:
p = h5py.File('datasets/test_vectors_tenK.hdf5', 'r')
test_preprocessed_x = np.array(p['test_preprocessed'])

In [ ]:
"""
Testing Linear Regression on the test set
"""
lr = linear_model.LogisticRegression(C=1e-06)
lr =lr.fit(train_x, train_y)
predicted_Val = lr.predict(test_preprocessed_x)

In [ ]:
#saving the new predicted labels in a csv that is to be submitted on Kaggle
df = pd.DataFrame(predicted_Val, columns=['Category'])
df.to_csv('kaggle_test.csv', index_label='Id')